In [ ]:
import datetime
import os
import numpy as np
import seaborn as sns
import vaex
import pandas as pd

from math import pi
from matplotlib_venn import venn3
from matplotlib import pyplot as plt
from tqdm import tqdm

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import resample

vaex.settings.main.home = '/data/vaex'

# Utilities

Utility functions for computing percentages and plotting circle progress plots.

In [ ]:
def get_percentage(subset_size, total_size):
    return round(100 * subset_size / total_size, 2)

In [ ]:
def plot_progress(percentage, color, title):
    fig, ax = plt.subplots(figsize=(2, 2), subplot_kw={'projection': 'polar'}, dpi=100)
    start_angle = 90
    percentage_rad = 2 * pi * (percentage / 100)
    start_rad = 2 * pi * start_angle / 360
    
    plt.xticks([])
    plt.yticks([])
    ax.spines.clear()
    
    ax.barh(1, percentage_rad, left=start_rad, height=1, color=color) 
    plt.ylim(-3, 3)
    plt.text(0, -3, f"{percentage}%", ha='center', va='center', fontsize=15)
    plt.title(title)
    plt.show()

# Data Loading

Loads the key tables from MIMIC-IV. Uses `open` for larger tables to conserve RAM (`open` reads the source file lazily) and `read_csv` for smaller tables (could be further optimised by specifying `convert=True` which converts the CSV to HDF5 after loading and frees the associated memory, but this is not currently needed). See the [I/O Kung-Fu](https://vaex.readthedocs.io/en/latest/guides/io.html) page in Vaex documentation for details on I/O operations.

In [ ]:
# Load the patients from the MIMIC-IV database
patients_df = vaex.open('mimic-iv-2.2/hosp/patients.csv')

In [ ]:
# Load the data for hospital admissions
admissions_df = vaex.open('mimic-iv-2.2/hosp/admissions.csv')

In [ ]:
# Load the CXR metadata for the patients in the MIMIC-IV database
cxr_df = vaex.open('mimic-cxr-2.0.0-metadata.csv')
cxr_df = cxr_df[cxr_df.subject_id.isin(patients_df.subject_id.unique())]

In [ ]:
# Load the note data for the patients in the MIMIC-IV database
discharge_meta_df = vaex.read_csv('mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/discharge.csv')
discharge_meta_df.drop(columns=["text"], inplace=True)
discharge_meta_df = discharge_meta_df[discharge_meta_df.subject_id.isin(patients_df.subject_id.unique())]
radiology_meta_df = vaex.read_csv('mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/radiology.csv')
radiology_meta_df.drop(columns=["text"], inplace=True)
radiology_meta_df = radiology_meta_df[radiology_meta_df.subject_id.isin(patients_df.subject_id.unique())]

In [ ]:
# Load the ICU data for the patients in the MIMIC-IV database
chartevents_df = vaex.open('mimic-iv-2.2/icu/chartevents.csv')
icustays_df = vaex.open('mimic-iv-2.2/icu/icustays.csv')
items_df = vaex.read_csv('mimic-iv-2.2/icu/d_items.csv')

In [ ]:
# Load the ICD diagnoses data for the patients in the MIMIC-IV database
icd_diagnoses_codes = vaex.open('mimic-iv-2.2/hosp/d_icd_diagnoses.csv')
icd_diagnoses = vaex.open('mimic-iv-2.2/hosp/diagnoses_icd.csv')

## Modality Overlap

### All patients
Shows the number of unique patients from the entire MIMIC-IV dataset for which each of the different data modalities (CXR image, radiology note and discharge note) is available.

In [ ]:
num_core = len(patients_df.subject_id.unique())
patients_cxr = set(cxr_df.subject_id.unique())
patients_discharge = set(discharge_meta_df.subject_id.unique())
patients_radiology = set(radiology_meta_df.subject_id.unique())

only_cxr = len(patients_cxr - patients_discharge - patients_radiology)
only_discharge = len(patients_discharge - patients_cxr - patients_radiology)
only_radiology = len(patients_radiology - patients_cxr - patients_discharge)
cxr_discharge = len((patients_cxr & patients_discharge) - patients_radiology)
cxr_radiology = len((patients_cxr & patients_radiology) - patients_discharge)
discharge_radiology = len((patients_discharge & patients_radiology) - patients_cxr)
all_data = len(patients_cxr & patients_discharge & patients_radiology)

print(f"Total MIMIC-IV patients: {num_core}")
print(f"Total patients with at least one additional modality: {len(patients_cxr | patients_discharge | patients_radiology)}")
venn3(
    subsets=(only_cxr, only_discharge, cxr_discharge, only_radiology, cxr_radiology, discharge_radiology, all_data),
    set_labels=("CXR", "Discharge", "Radiology")
)
plt.show()

### ICU Patients
Shows similar statistic as the cell above, but restricts the set of patients to those with ICU data available.

In [ ]:
num_core = len(patients_df.subject_id.unique())
patients_icu = set(chartevents_df.subject_id.unique())
num_icu = len(patients_icu)
patients_cxr = set(cxr_df.subject_id.unique()) & patients_icu
patients_discharge = set(discharge_meta_df.subject_id.unique()) & patients_icu
patients_radiology = set(radiology_meta_df.subject_id.unique()) & patients_icu

only_cxr = len(patients_cxr - patients_discharge - patients_radiology)
only_discharge = len(patients_discharge - patients_cxr - patients_radiology)
only_radiology = len(patients_radiology - patients_cxr - patients_discharge)
cxr_discharge = len((patients_cxr & patients_discharge) - patients_radiology)
cxr_radiology = len((patients_cxr & patients_radiology) - patients_discharge)
discharge_radiology = len((patients_discharge & patients_radiology) - patients_cxr)
all_data = len(patients_cxr & patients_discharge & patients_radiology)

print(f"Total MIMIC-IV patients: {num_core}")
print(f"Total MIMIC-IV ICU patients: {num_icu}")
print(f"Total patients with at least one additional modality: {len(patients_cxr | patients_discharge | patients_radiology)}")
venn3(
    subsets=(only_cxr, only_discharge, cxr_discharge, only_radiology, cxr_radiology, discharge_radiology, all_data),
    set_labels=("CXR", "Discharge", "Radiology")
)
plt.show()

## Data Completeness
Displays the most common values in the `chartevents` ICU table and the associated information, including the typical sampling rate per 24 hours (`rate`) and the number of unique ICU patients for which at least one such value was recorded (`subject_id_nunique`). The produced table only displays information for values recorded for at least $10000$ patients (roughly $20\%$ of all ICU patients).

In [ ]:
event_counts_df = chartevents_df.groupby("itemid").agg('count')
event_counts_df = event_counts_df.join(items_df, left_on='itemid', right_on='itemid')
event_counts_df.drop(columns=["linksto", "lownormalvalue", "highnormalvalue"], inplace=True)
patient_event_counts_df = chartevents_df[["itemid", "subject_id"]].groupby("itemid").agg('nunique')
event_counts_df["rate"] = event_counts_df["count"] / icustays_df["los"].sum()
event_counts_df = event_counts_df.join(patient_event_counts_df, left_on='itemid', right_on='itemid')
event_counts_df = event_counts_df[event_counts_df["subject_id_nunique"] > 10000]

In [ ]:
event_counts_df = event_counts_df.sort(by="subject_id_nunique", ascending=False)
vaex.settings.display.max_rows = 700
event_counts_df

## ICD Diagnoses
Shows the $100$ most common ICD-10 diagnoses assigned at the end of each hospital stay for billing purposes. Note that there could be up to $39$ diagnoses for each hospital stay. The `seq_num` column in the `diagnoses_icd` table provides the approximate ordering of diagnoses in terms of their "importance", but this information is considered unreliable (the billing department is not particularly incentivised to enter a highly accurate ranking and there are certain specific rules for some conditions, e.g. sepsis should always be ranked second with the associated infectious agent ranked first). The code below groups diagnoses by the first three digits of the ICD-10 code in order to obtain more general categories.

**TODO:** The code below only considers patients for which diagnosis was entered using ICD-10 code. The diagnoses information for many patients was entered using the older ICD-9 system — we should convert the ICD-9 codes to ICD-10 and then compute the statistics for a more accurate result.

In [ ]:
# TODO: Convert ICD-9 codes
icd_10_icu_patients = icd_diagnoses[icd_diagnoses["subject_id"].isin(list(patients_icu))][icd_diagnoses["icd_version"] == 10]
icd_10_icu_patients["icd_code"] = icd_10_icu_patients.apply(lambda code: code[0:3], [icd_10_icu_patients.icd_code])
icd_10_counts = icd_10_icu_patients.groupby("icd_code").agg('count').sort(by="count", ascending=False)
icd_10_counts = icd_10_counts.join(icd_diagnoses_codes[icd_diagnoses_codes["icd_version"] == 10], left_on="icd_code", right_on="icd_code")

In [ ]:
vaex.settings.display.max_rows = 100
icd_10_counts[0:100]

## Mortality

### In-Hospital

In [ ]:
total_admitted = admissions_df.subject_id.nunique()
died_in_hospital = admissions_df[admissions_df["hospital_expire_flag"] == 1].subject_id.nunique()
mortality_rate = np.round(100 * (died_in_hospital / total_admitted), 1)
print(f"{died_in_hospital}/{total_admitted} ({mortality_rate}%) of all admitted patients died in hospital")
plot_progress(mortality_rate, "cyan", "In-Hospital Mortality (All)")

admissions_with_icu_df = admissions_df[admissions_df["subject_id"].isin(chartevents_df.subject_id.unique())]
total_admitted = admissions_with_icu_df.subject_id.nunique()
died_in_hospital = admissions_with_icu_df[admissions_with_icu_df["hospital_expire_flag"] == 1].subject_id.nunique()
mortality_rate = np.round(100 * (died_in_hospital / total_admitted), 1)
print(f"{died_in_hospital}/{total_admitted} ({mortality_rate}%) of ICU patients died in hospital")
plot_progress(mortality_rate, "cyan", "In-Hospital Mortality (ICU)")

### Total
**Note:** Out-of-hospital mortality data are censored one year after the last hospital visit for each patient.

In [ ]:
total_patients = patients_df.subject_id.nunique()
died = patients_df[~patients_df["dod"].ismissing()].subject_id.nunique()
mortality_rate = np.round(100 * (died / total_patients), 1)
print(f"{died}/{total_patients} ({mortality_rate}%) of all patients died")
plot_progress(mortality_rate, "cyan", "Total Mortality (All)")

patients_with_icu_df = patients_df[patients_df["subject_id"].isin(chartevents_df.subject_id.unique())]
total_patients = patients_with_icu_df.subject_id.nunique()
died = patients_df[~patients_with_icu_df["dod"].ismissing()].subject_id.nunique()
mortality_rate = np.round(100 * (died / total_patients), 1)
print(f"{died}/{total_patients} ({mortality_rate}%) of all ICU patients died")
plot_progress(mortality_rate, "cyan", "Total Mortality (ICU)")

## Age

In [ ]:
def compute_age_statistics(admissions_df, patient_group, show=False):
    admissions_anchors_df = admissions_df.join(patients_df[["subject_id", "anchor_year", "anchor_age"]], left_on="subject_id", right_on="subject_id")
    admissions_anchors_df["admitage"] = admissions_anchors_df["admittime"].dt.year - admissions_anchors_df["anchor_year"] + admissions_anchors_df["anchor_age"]
    admissions_anchors_df["admitage"] = admissions_anchors_df.apply(lambda age: min(age, 91), [admissions_anchors_df.admitage])
    patient_ages_series = admissions_anchors_df["admitage"]
    patient_ages_list = admissions_anchors_df["admitage"].tolist()
    
    print(f"Age statistics for {patient_group}")
    print(f"Mean: {patient_ages_series.mean()}, std: {patient_ages_series.std()}")
    print(f"Quantiles [0.25, 0.5, 0.75]: {[np.quantile(patient_ages_list, q) for q in [0.25, 0.5, 0.75]]}")
    sns.histplot(data=patient_ages_list, binwidth=1, label=patient_group)
    plt.title(f"Patient Age Distribution")
    if show:
        plt.show()
    
compute_age_statistics(admissions_df, "All patients")
print()
admissions_with_icu_df = admissions_df[admissions_df["subject_id"].isin(chartevents_df.subject_id.unique())]
compute_age_statistics(admissions_with_icu_df, "ICU patients")
plt.show()

## Gender

In [ ]:
def autopct_format(values):
    def format_fun(pct):
        total = sum(values)
        absolute = int(pct / 100. * total + 0.5)
        return "{:.1f}%\n({:d})".format(pct, absolute)
    
    return format_fun

gender_counts = patients_df["gender"].value_counts()
plt.figure(figsize=(3, 3))
plt.pie(gender_counts, labels=gender_counts.index, colors=["pink", "cyan"], autopct=autopct_format(gender_counts), startangle=90)
plt.title("Patient Gender Distribution (All)")
plt.show()

patients_with_icu_df = patients_df[patients_df["subject_id"].isin(chartevents_df.subject_id.unique())]
gender_counts = patients_with_icu_df["gender"].value_counts()
plt.figure(figsize=(3, 3))
plt.pie(gender_counts, labels=gender_counts.index, colors=["cyan", "pink"], autopct=autopct_format(gender_counts), startangle=90)
plt.title("Patient Gender Distribution (ICU)")
plt.show()

# Dataset Generation

First, get the very first ICU stay record for each patient. For simplicity, we do not currently consider further ICU stays.

In [ ]:
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S.%f"

earliest_icu_stay_dates_df = icustays_df.groupby("subject_id").agg({"intime": "min"})

# Vaex is currently unable to join on multiple columns, so we construct an auxiliary column as a workaround
# Also, we use strftime instead of astype('str') to avoid a weird Vaex issue when converting datetime to string
earliest_icu_stay_dates_df["subject_id_intime"] = earliest_icu_stay_dates_df.subject_id.astype('str') + earliest_icu_stay_dates_df.intime.dt.strftime(DATETIME_FORMAT)
icustays_df["subject_id_intime"] = icustays_df.subject_id.astype('str') + icustays_df.intime.dt.strftime(DATETIME_FORMAT)

earliest_icu_stays_df = icustays_df.join(earliest_icu_stay_dates_df, left_on="subject_id_intime", right_on="subject_id_intime", rsuffix="_tmp", how="inner")
earliest_icu_stays_df.drop(columns=[
    "last_careunit", "los", "subject_id_intime", "subject_id_tmp", "intime_tmp", "subject_id_intime_tmp",
], inplace=True)
None

Join the ICU records with the available admissions data.

In [ ]:
filtered_admissions_df = admissions_df.drop(columns=[
    "admit_provider_id", "discharge_location", "insurance", "language", "marital_status",
    "edregtime", "edouttime"
])

filtered_stays_df = earliest_icu_stays_df.join(filtered_admissions_df, left_on="hadm_id", right_on="hadm_id", rsuffix="_icu", how="inner")

Add information from the patients table, compute the age of all patients and filter out the stays for patients with anonymised age (90 years and older).

In [ ]:
filtered_stays_age_df = filtered_stays_df.join(patients_df[["subject_id", "anchor_year", "anchor_age", "gender", "dod"]], left_on="subject_id", right_on="subject_id", how="inner")
filtered_stays_age_df["admitage"] = filtered_stays_age_df["admittime"].dt.year - filtered_stays_age_df["anchor_year"] + filtered_stays_age_df["anchor_age"]
filtered_stays_age_df["admitage"] = filtered_stays_age_df.apply(lambda age: min(age, 91), [filtered_stays_age_df.admitage])
filtered_stays_age_df = filtered_stays_age_df[filtered_stays_age_df["admitage"] < 91]

Compute the 30-day mortality flag to be used as a label (indicating whether the patient died within 30 days of being admitted to an ICU).

In [ ]:
filtered_stays_deaths = filtered_stays_age_df.copy()
filtered_stays_deaths["dod"] = filtered_stays_deaths.dod.astype("datetime64")
filtered_stays_deaths["time_to_death"] = filtered_stays_deaths["dod"] - filtered_stays_deaths["intime"]
filtered_stays_deaths["died_30_days"] = filtered_stays_deaths.apply(lambda ttd: ttd is not None and ttd < datetime.timedelta(days=30), [filtered_stays_deaths.time_to_death])

Add the medical features, measured at the start of the ICU stay (heart rate, SpO2, respiratory rate, weight, blood pressure mean, temperature)

In [ ]:
SELECTED_FEATURES = ["HR", "SpO2", "RR", "Admission Weight (Kg)", "NBPm", "Temperature F"]

selected_items_defs_df = items_df[items_df.abbreviation.isin(SELECTED_FEATURES)]
selected_items_defs_df.drop(columns=[
    "label", "linksto", "category", "unitname", "param_type", "lownormalvalue", "highnormalvalue"
])
selected_events_df = chartevents_df.join(selected_items_defs_df, left_on="itemid", right_on="itemid", how="inner")

In [ ]:
# Select only the first measurement of each feature per ICU stay
# This operation is a bit lengthy, so it's being performed in a separate cell
earliest_icu_event_dates_df = selected_events_df.groupby(["stay_id", "itemid"]).agg({"charttime": "min"})

In [ ]:
# We use a similar trick for joining on multiple columns as above
earliest_icu_event_dates_df["stay_id_itemid_charttime"] = earliest_icu_event_dates_df.stay_id.astype('str') + earliest_icu_event_dates_df.itemid.astype('str') + earliest_icu_event_dates_df.charttime.dt.strftime(DATETIME_FORMAT)
selected_events_df["stay_id_itemid_charttime"] = selected_events_df.stay_id.astype('str') + selected_events_df.itemid.astype('str') + selected_events_df.charttime.dt.strftime(DATETIME_FORMAT)

earliest_selected_events_df = selected_events_df.join(earliest_icu_event_dates_df, left_on="stay_id_itemid_charttime", right_on="stay_id_itemid_charttime", rsuffix="_tmp", how="inner")
earliest_selected_events_df.drop(columns=[
    "caregiver_id", "storetime", "value", "warning", "label", "linksto", "category", "unitname", "param_type",
    "lownormalvalue", "highnormalvalue", "stay_id_itemid_charttime", "stay_id_tmp", "itemid_tmp", "charttime_tmp",
    "stay_id_itemid_charttime_tmp", "subject_id", "hadm_id", "charttime", "itemid", "valueuom"
], inplace=True)
earliest_selected_events_df = earliest_selected_events_df.join(filtered_stays_deaths[["stay_id"]], left_on="stay_id", right_on="stay_id", how="inner", rsuffix="_tmp")
earliest_selected_events_df.drop(columns="stay_id_tmp", inplace=True)
None

In [ ]:
# Now, we need to transpose the events data frame so that the medical features are
# in different columns rather than different rows. We do this operation in Pandas
# as Vaex doesn't support transforming the shape of the data frame.

FEATURE_RENAME_TABLE = {
    "Admission Weight (Kg)": "admission_weight",
    "HR": "hr",
    "NBPm": "mbp",
    "RR": "rr",
    "SpO2": "spo2",
    "Temperature F": "temperature",
    "": "stay_id",
}

earliest_selected_events_pd_df = earliest_selected_events_df.to_pandas_df()
medical_features_pd_df = earliest_selected_events_pd_df.pivot(index="stay_id", columns="abbreviation")

# Reset the index and drop its topmost level
medical_features_pd_df = medical_features_pd_df.reset_index().droplevel(0, axis=1).dropna().rename_axis(None, axis=1)
medical_features_pd_df.rename(columns=FEATURE_RENAME_TABLE, inplace=True)
medical_features_df = vaex.from_pandas(medical_features_pd_df)

In [ ]:
# Finally, we join the medical features with the rest of the data
data_df = medical_features_df.join(filtered_stays_deaths, left_on="stay_id", right_on="stay_id", how="inner")
data_df.drop(columns=[
    "intime", "outtime", "admittime", "dischtime", "deathtime", "hospital_expire_flag", "anchor_year",
    "anchor_age", "dod", "time_to_death"
], inplace=True)
None

# Additional Processing

## Initial Preview
Display a preview of the dataset

In [ ]:
data_df

Print basic statistics about the data (IDs and non-numeric features omitted here)

In [ ]:
NUMERIC_FEATURES = ["admission_weight", "hr", "mbp", "rr", "spo2", "temperature", "admitage"]
data_df[NUMERIC_FEATURES].describe()

## Data Cleaning
Filter data to exclude records with implausible values.

In [ ]:
NUMERIC_FEATURE_RANGES = {
    "admission_weight": (3, 650),
    "hr": (25, 480),
    "mbp": (20, 250),
    "rr": (1, 100),
    "spo2": (40, 100),
    "temperature": (40, 115),
    "admitage": (18, 100)
}

for feature, (minimum, maximum) in NUMERIC_FEATURE_RANGES.items():
    data_df = data_df[data_df[feature] >= minimum]
    data_df = data_df[data_df[feature] <= maximum]
    
data_df.drop(columns=[
    "stay_id", "subject_id", "hadm_id", "first_careunit", "subject_id_icu", "hadm_id_icu", "admission_location", "race"
], inplace=True)
None

## Final Statistics
Show the final data statistics

In [ ]:
data_df

In [ ]:
data_df[NUMERIC_FEATURES].describe()

In [ ]:
died = len(data_df[data_df["died_30_days"]])
total = len(data_df)
mortality_rate = get_percentage(died, total)
print(f"{died}/{total} ({mortality_rate}%) of all patients died")
plot_progress(mortality_rate, "cyan", "30-day Patient Mortality")

In [ ]:
def autopct_format(values):
    def format_fun(pct):
        total = sum(values)
        absolute = int(pct / 100. * total + 0.5)
        return "{:.1f}%\n({:d})".format(pct, absolute)
    
    return format_fun

gender_counts = data_df["gender"].value_counts()
plt.figure(figsize=(3, 3))
plt.pie(gender_counts, labels=gender_counts.index, colors=["cyan", "pink"], autopct=autopct_format(gender_counts), startangle=90)
plt.title("Patient Gender Distribution")
plt.show()

In [ ]:
data_df["admission_type"].value_counts()

In [ ]:
sns.heatmap(data_df.drop(columns=["admission_type", "gender"]).to_pandas_df().corr(numeric_only=False), annot=True)

In [ ]:
data_df.export('./simple_mimic.csv')

## Final Data Processing

In [ ]:
data_df = vaex.read_csv('./simple_mimic.csv')